In [13]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [14]:
companies = [
    "BHARTIARTL",
    "ITC",
    "KOTAKBANK",
    "HINDUNILVR",
    "ULTRACEMCO",
    "TITAN",
    "POWERGRID",
    "HCLTECH",
    "NESTLEIND",
    "GRASIM",
    "ADANIPORTS",
    "ICICIBANK",
    "HDFCLIFE",
    "SBIN",
    "MARUTI",
    "RELIANCE",
    "ADANIENT",
    "BEL",
    "ASIANPAINT",
    "TATACONSUM",
    "LT",
    "HDFCBANK",
    "BAJFINANCE",
    "BAJAJ-AUTO",
    "NTPC",
    "INFY",
    "TATAMOTORS",
    "AXISBANK",
    "TECHM",
    "M&M",
    "COALINDIA",
    "EICHERMOT",
    "TCS",
    "APOLLOHOSP",
    "HEROMOTOCO",
    "ONGC",
    "WIPRO",
    "SUNPHARMA",
    "BRITANNIA",
    "CIPLA",
    "DRREDDY",
    "BAJAJFINSV",
    "BPCL",
    "SBILIFE",
    "TRENT",
    "HINDALCO",
    "JSWSTEEL",
    "INDUSINDBK",
    "TATASTEEL",
    "SHRIRAMFIN"
]


In [15]:
def ticker_info(ticker):
  r = requests.get(f'https://www.screener.in/company/{ticker}/')
  soup = BeautifulSoup(r.content, 'html.parser')

  top_sec = soup.find("div", {"id": "top"})
  pnl_sec = soup.find("section", {"id": "profit-loss"})

  ratios = top_sec.find("ul", {"id": "top-ratios"})
  ratios = ratios.find_all("li")
  ratios = ratios[0:4]

  values = []
  for ratio in ratios:
    for elements in ratio.find_all("span",{"class": "number"}):
      values.append(elements.text)

  col_names = ["MarketCap", "LTP", "High", "Low", "PE"]

  ratios_dict = {}

  for i in range(len(col_names)):
    ratios_dict[col_names[i]] =values[i]

  input_string = top_sec.find("span", {"class":"font-size-12"}).get_text()
  match = re.search(r"-?\s*\d+\.\d+", input_string)
  if match:
    ratios_dict["PercentChange"] = float(match.group().replace(" ", ""))

  ratios_dict["EPS"] = pnl_sec.find_all("tr")[11].find_all("td")[12].text

  return ratios_dict


In [24]:
def stock_returns(ticker):

    end_date = datetime.now()

    periods = {
        '6M': end_date - timedelta(days=180),
        '1Y': end_date - timedelta(days=365),
        '3Y': end_date - timedelta(days=365*3),
        '5Y': end_date - timedelta(days=365*5)
    }

    stock = yf.Ticker(ticker)

    returns = {}

    for period_name, start_date in periods.items():
        hist_data = stock.history(start=start_date, end=end_date)

        if len(hist_data) == 0:
          returns[period_name] = None

        start_price = hist_data['Close'].iloc[0]
        end_price = hist_data['Close'].iloc[-1]

        return_pct = ((end_price - start_price) / start_price) * 100
        returns[period_name] = round(return_pct, 2)

        if period_name == "1Y":
          returns["volume"] = hist_data['Volume'].mean()

    return returns


In [37]:
df = []
for company in companies:
  ratios_details = ticker_info(company)
  return_details = stock_returns(f'{company}.NS')
  df.append({"ticker": company, **ratios_details,**return_details})

In [38]:
df = pd.DataFrame(df)
df.set_index("ticker", inplace=True)

In [39]:
df

,MarketCap,LTP,High,Low,PE,PercentChange,EPS,6M,1Y,volume,3Y,5Y
ticker,,,,,,,,,,,,
BHARTIARTL,"10,07,452","1,682","1,779",960,133,4.42,8.82,18.40,70.38,6.687784e+06,148.95,313.59
ITC,"5,88,008",470,529,399,28.6,2.04,16.36,9.62,5.93,1.520944e+07,133.93,144.75
KOTAKBANK,"3,58,993","1,806","1,953","1,544",25.6,2.09,69.33,5.13,-2.32,5.703585e+06,-3.06,6.29
HINDUNILVR,"5,61,576","2,390","3,035","2,170",55.3,1.93,43.05,-3.14,-3.61,1.967230e+06,8.06,30.99
ULTRACEMCO,"3,48,860","12,084","12,138","9,250",53.7,1.91,239.18,9.32,21.20,3.563856e+05,66.61,204.83
TITAN,"3,11,511","3,509","3,887","3,056",94.4,1.83,39.92,-1.91,-2.23,1.229415e+06,53.08,208.15
POWERGRID,"3,10,501",334,366,223,20.4,1.41,16.64,2.91,46.27,1.789673e+07,154.30,329.31
HCLTECH,"5,34,266","1,969","1,975","1,235",45.2,1.68,43.02,38.94,36.75,3.138499e+06,94.24,320.00
NESTLEIND,"2,17,273","2,254","2,778","2,169",68.4,1.32,40.79,-11.26,-6.85,9.780754e+05,20.78,67.69
